In [1]:
print(1)

1


In [5]:
import pandas as pd


In [6]:
file_path = "output/techinasia_ai_news_batch_0_20250116_174236.csv"

In [7]:
df = pd.read_csv(file_path)

In [10]:
df['article_url']

0     https://www.techinasia.com/news/chinese-unicor...
1     https://www.techinasia.com/news/meta-exec-aime...
2     https://www.techinasia.com/news/nvidia-build-a...
3     https://www.techinasia.com/news/nvidia-backed-...
4     https://www.techinasia.com/news/tsmc-q4-profit...
5     https://www.techinasia.com/news/chinese-ai-chi...
6     https://www.techinasia.com/news/israel-boost-a...
7     https://www.techinasia.com/news/israeli-hospit...
8     https://www.techinasia.com/news/google-adds-ai...
9     https://www.techinasia.com/news/afp-news-agenc...
10    https://www.techinasia.com/news/linkedin-launc...
11    https://www.techinasia.com/news/openai-unveils...
12    https://www.techinasia.com/news/infrastructure...
13    https://www.techinasia.com/news/openai-launche...
14    https://www.techinasia.com/news/bytedance-plan...
15    https://www.techinasia.com/news/ytl-sea-launch...
16    https://www.techinasia.com/news/taiwanese-star...
17    https://www.techinasia.com/news/saudi-firm

In [23]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

async def fetch_article_content(url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto(url, timeout=10000)
            await page.wait_for_selector("div#content.content", timeout=10000)
            soup = BeautifulSoup(await page.content(), "html.parser")
            content = soup.select_one("div#content.content")
            return content.get_text(strip=True) if content else None
        finally:
            await browser.close()

async def main():
    url = "https://www.techinasia.com/news/chinese-unicorn-minimax-unveils-new-ai-models"
    content = await fetch_article_content(url)
    print(content)

if __name__ == "__main__":
    asyncio.run(main())


Shanghai-based AI startup MiniMax has released a new family of open-source large language models (LLMs), called MiniMax-01.The launch includes the general-purpose MiniMax-Text-01 model and the multimodal MiniMax-VL-01, which integrates visual capabilities along with text processing.According to benchmark tests shared on MiniMax’s official WeChat account, the foundational language model aligns with global standards in areas such as mathematics, specialized knowledge, instruction-following, and reducing factual errors.


In [ ]:
# now for each of the article_url in the dataframe, we can fetch the content. put them in a structured json. please note that we shall introduce a delay between each fetch to avoid being blocked by the website. and introduce randomness and shuffle the user agents to avoid being detected as a bot.



In [24]:
import asyncio
import random
import time
from tqdm import tqdm
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import json

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1"
]

async def fetch_article_content_with_delay(url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True)
        page = await browser.new_page(user_agent=random.choice(user_agents))
        try:
            await page.goto(url, timeout=10000)
            await page.wait_for_selector("div#content.content", timeout=10000)
            soup = BeautifulSoup(await page.content(), "html.parser")
            content = soup.select_one("div#content.content")
            return content.get_text(strip=True) if content else None
        finally:
            await browser.close()

async def fetch_all_articles(df):
    data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        url = row['article_url']
        content = await fetch_article_content_with_delay(url)
        if content:
            data.append({"article_id": row['article_id'], "title": row['title'], "url": url, "content": content})
        time.sleep(random.uniform(2, 5))  # Introduce delay with randomness
    with open("articles.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    asyncio.run(fetch_all_articles(df))

100%|██████████| 50/50 [06:47<00:00,  8.16s/it]


In [ ]:
import asyncio
import random
import time
from tqdm import tqdm
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import json

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1"
]

async def fetch_article_content_with_delay(url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True)
        page = await browser.new_page(user_agent=random.choice(user_agents))
        try:
            await page.goto(url, timeout=10000)
            await page.wait_for_selector("div#content.content", timeout=10000)
            soup = BeautifulSoup(await page.content(), "html.parser")
            content = soup.select_one("div#content.content")
            return content.get_text(strip=True) if content else None
        finally:
            await browser.close()

async def fetch_all_articles(df):
    data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        url = row['article_url']
        content = await fetch_article_content_with_delay(url)
        if content:
            data.append({"article_id": row['article_id'], "title": row['title'], "url": url, "content": content})
        time.sleep(random.uniform(1, 3))  # Introduce delay with randomness
    with open("articles.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    asyncio.run(fetch_all_articles(df))

In [ ]:
import asyncio
import random
import time
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

async def fetch_article_content_with_randomization(url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True)
        viewport_sizes = [
            {"width": 1920, "height": 1080},
            {"width": 1366, "height": 768},
            {"width": 1280, "height": 720},
            {"width": 1440, "height": 900}
        ]
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1"
        ]
        page = await browser.new_page(
            user_agent=random.choice(user_agents),
            viewport=random.choice(viewport_sizes)
        )
        try:
            await page.goto(url, timeout=10000)
            await page.wait_for_selector("div#content.content", timeout=10000)

            # Simulate random scrolling behavior
            for _ in range(random.randint(3, 8)):
                scroll_position = random.randint(100, 500)
                await page.evaluate(f"window.scrollBy(0, {scroll_position})")
                await asyncio.sleep(random.uniform(0.5, 1.5))

            # Simulate random mouse movements
            for _ in range(random.randint(5, 10)):
                x, y = random.randint(0, 1920), random.randint(0, 1080)
                await page.mouse.move(x, y, steps=random.randint(1, 5))
                await asyncio.sleep(random.uniform(0.2, 0.8))

            soup = BeautifulSoup(await page.content(), "html.parser")
            content = soup.select_one("div#content.content")
            return content.get_text(strip=True) if content else None
        finally:
            await browser.close()

async def fetch_all_articles(df):
    data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        url = row["article_url"]
        content = await fetch_article_content_with_randomization(url)
        if content:
            data.append({
                "article_id": row["article_id"],
                "title": row["title"],
                "url": url,
                "content": content
            })
        time.sleep(random.uniform(1, 3))  # Introduce delay with randomness
    with open("articles.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    asyncio.run(fetch_all_articles(df))

In [25]:
import logging

# Configure logging
logging.basicConfig(filename="errors.log", level=logging.ERROR,
                    format="%(asctime)s - %(levelname)s - %(message)s")

async def fetch_article_content_with_randomization(url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True)
        viewport_sizes = [
            {"width": 1920, "height": 1080},
            {"width": 1366, "height": 768},
            {"width": 1280, "height": 720},
            {"width": 1440, "height": 900}
        ]
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1"
        ]
        try:
            page = await browser.new_page(
                user_agent=random.choice(user_agents),
                viewport=random.choice(viewport_sizes)
            )
            await page.goto(url, timeout=10000)
            await page.wait_for_selector("div#content.content", timeout=10000)

            # Simulate random scrolling behavior
            for _ in range(random.randint(3, 8)):
                scroll_position = random.randint(100, 500)
                await page.evaluate(f"window.scrollBy(0, {scroll_position})")
                await asyncio.sleep(random.uniform(0.5, 1.5))

            # Simulate random mouse movements
            for _ in range(random.randint(5, 10)):
                x, y = random.randint(0, 1920), random.randint(0, 1080)
                await page.mouse.move(x, y, steps=random.randint(1, 5))
                await asyncio.sleep(random.uniform(0.2, 0.8))

            soup = BeautifulSoup(await page.content(), "html.parser")
            content = soup.select_one("div#content.content")
            return content.get_text(strip=True) if content else None
        except Exception as e:
            logging.error(f"Error fetching URL {url}: {e}")
            return None
        finally:
            await browser.close()

async def fetch_all_articles(df):
    data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        url = row["article_url"]
        content = await fetch_article_content_with_randomization(url)
        if content:
            data.append({
                "article_id": row["article_id"],
                "title": row["title"],
                "url": url,
                "content": content
            })
        time.sleep(random.uniform(1, 3))  # Introduce delay with randomness

    timestamp_str = time.strftime("%Y%m%d_%H%M%S")
    filename = f"articles_{timestamp_str}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    asyncio.run(fetch_all_articles(df))

 18%|█▊        | 9/50 [02:32<11:33, 16.91s/it]


KeyboardInterrupt: 

In [26]:
import logging

# Adjustable parameters
SCROLL_ITERATIONS_RANGE = (2, 4)
SCROLL_DISTANCE_RANGE = (50, 200)
MOUSE_MOVEMENTS_RANGE = (2, 5)
SLEEP_SCROLL_RANGE = (0.3, 0.8)
SLEEP_MOUSE_RANGE = (0.1, 0.3)
URL_DELAY_RANGE = (0.5, 1)

# Configure logging
logging.basicConfig(filename="errors.log", level=logging.ERROR,
                    format="%(asctime)s - %(levelname)s - %(message)s")

async def fetch_article_content_with_randomization(url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True)
        viewport_sizes = [
            {"width": 1920, "height": 1080},
            {"width": 1366, "height": 768},
            {"width": 1280, "height": 720},
            {"width": 1440, "height": 900}
        ]
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59",
        ]
        try:
            page = await browser.new_page(
                user_agent=random.choice(user_agents),
                viewport=random.choice(viewport_sizes)
            )
            await page.goto(url, timeout=10000)
            await page.wait_for_selector("div#content.content", timeout=10000)

            # Simulate scrolling behavior
            for _ in range(random.randint(*SCROLL_ITERATIONS_RANGE)):
                scroll_position = random.randint(*SCROLL_DISTANCE_RANGE)
                await page.evaluate(f"window.scrollBy(0, {scroll_position})")
                await asyncio.sleep(random.uniform(*SLEEP_SCROLL_RANGE))

            # Simulate mouse movements
            for _ in range(random.randint(*MOUSE_MOVEMENTS_RANGE)):
                x, y = random.randint(0, 1920), random.randint(0, 1080)
                await page.mouse.move(x, y, steps=random.randint(1, 3))
                await asyncio.sleep(random.uniform(*SLEEP_MOUSE_RANGE))

            soup = BeautifulSoup(await page.content(), "html.parser")
            content = soup.select_one("div#content.content")
            return content.get_text(strip=True) if content else None
        except Exception as e:
            logging.error(f"Error fetching URL {url}: {e}")
            return None
        finally:
            await browser.close()

async def fetch_all_articles(df):
    data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        url = row["article_url"]
        content = await fetch_article_content_with_randomization(url)
        if content:
            data.append({
                "article_id": row["article_id"],
                "title": row["title"],
                "url": url,
                "content": content
            })
        time.sleep(random.uniform(*URL_DELAY_RANGE))  # Delay between requests

    timestamp_str = time.strftime("%Y%m%d_%H%M%S")
    filename = f"articles_{timestamp_str}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    asyncio.run(fetch_all_articles(df))


 18%|█▊        | 9/50 [01:19<05:25,  7.94s/it]/usr/local/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/events.py:36: RuntimeWarning: coroutine 'fetch_article_content' was never awaited
  def __init__(self, callback, args, loop, context=None):
100%|██████████| 50/50 [06:22<00:00,  7.65s/it]


In [27]:
import asyncio
import random
import logging
import time
import json
import os
from playwright.async_api import async_playwright, TimeoutError
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd


# Adjustable parameters
SCROLL_ITERATIONS_RANGE = (2, 4)
SCROLL_DISTANCE_RANGE = (50, 200)
MOUSE_MOVEMENTS_RANGE = (2, 5)
SLEEP_SCROLL_RANGE = (0.3, 0.8)
SLEEP_MOUSE_RANGE = (0.1, 0.3)
URL_DELAY_RANGE = (0.5, 1)
RETRY_ATTEMPTS = 2


# Configure logging
logging.basicConfig(filename="errors.log", level=logging.ERROR,
                    format="%(asctime)s - %(levelname)s - %(message)s")

async def fetch_article_content_with_randomization(url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True) #  Consider passing a config value
        viewport_sizes = [
            {"width": 1920, "height": 1080},
            {"width": 1366, "height": 768},
            {"width": 1280, "height": 720},
            {"width": 1440, "height": 900}
        ]
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59",
        ]

        try:
            page = await browser.new_page(
                user_agent=random.choice(user_agents),
                viewport=random.choice(viewport_sizes)
            )
            await page.goto(url, timeout=10000)

            await page.wait_for_selector("div#content.content", timeout=10000)


            # Simulate scrolling behavior
            for _ in range(random.randint(*SCROLL_ITERATIONS_RANGE)):
                scroll_position = random.randint(*SCROLL_DISTANCE_RANGE)
                await page.evaluate(f"window.scrollBy(0, {scroll_position})")
                await asyncio.sleep(random.uniform(*SLEEP_SCROLL_RANGE))

            # Simulate mouse movements
            for _ in range(random.randint(*MOUSE_MOVEMENTS_RANGE)):
                x, y = random.randint(0, 1920), random.randint(0, 1080)
                await page.mouse.move(x, y, steps=random.randint(5, 10))
                await asyncio.sleep(random.uniform(*SLEEP_MOUSE_RANGE))

            await asyncio.sleep(random.uniform(0.5, 1.0)) # Wait for dynamic content to load

            soup = BeautifulSoup(await page.content(), "html.parser")
            content_selectors = [ # add fallback selectors if any
                "div#content.content",
            ]

            content = None
            for selector in content_selectors:
                content = soup.select_one(selector)
                if content:
                    break


            return content.get_text(strip=True) if content else None

        except TimeoutError as te:
            logging.error(f"Timeout error fetching URL {url}: {te}")
            return None

        except Exception as e:
            logging.error(f"Error fetching URL {url}: {e}")
            return None
        finally:
            if browser:
                await browser.close()


async def fetch_all_articles(df):
    data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        url = row["article_url"]

        for attempt in range(RETRY_ATTEMPTS):
            content = await fetch_article_content_with_randomization(url)
            if content:
                data.append({
                    "article_id": row["article_id"],
                    "title": row["title"],
                    "url": url,
                    "content": content
                })
                break # If successful break out of retry loop
            else:
                if attempt < RETRY_ATTEMPTS - 1:
                    await asyncio.sleep(random.uniform(1, 3) * (attempt + 1)) # Exponential backoff
                else:
                    logging.error(f"Failed to fetch URL {url} after {RETRY_ATTEMPTS} attempts.") # log error after all retries fail
        await asyncio.sleep(random.uniform(*URL_DELAY_RANGE))  # Delay between requests

    timestamp_str = time.strftime("%Y%m%d_%H%M%S")
    filename = f"articles_{timestamp_str}.json"
    filepath = os.path.join(".", filename)  # To ensure platform compatibility
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)



asyncio.run(fetch_all_articles(df))

 32%|███▏      | 16/50 [02:13<04:43,  8.32s/it]


KeyboardInterrupt: 

g

In [ ]:
import asyncio
import random
import logging
import time
import json
import os
from playwright.async_api import async_playwright, TimeoutError
from bs4 import BeautifulSoup
from tqdm.asyncio import tqdm
import pandas as pd
from typing import List, Dict, Optional

# --- Configuration ---
class Config:
    # --- Scrolling Simulation ---
    SCROLL_ITERATIONS_RANGE = (1, 3)
    # The range (min, max) for the number of times to simulate scrolling on a page.
    # This helps mimic human-like behavior and avoid detection.

    SCROLL_DISTANCE_RANGE = (50, 200)
    # The range (min, max) for the distance (in pixels) to scroll on each scroll iteration.
    # This simulates varying scroll lengths, making the bot less predictable.

    # --- Mouse Movement Simulation ---
    MOUSE_MOVEMENTS_RANGE = (1, 3)
    # The range (min, max) for the number of times to simulate mouse movements on a page.
    # This adds another layer of human-like interaction to avoid detection.

    # --- Sleep Durations ---
    SLEEP_SCROLL_RANGE = (0.2, 0.5)
    # The range (min, max) for the duration (in seconds) to sleep after each scroll action.
    # This simulates the time a human might take to read and scroll.

    SLEEP_MOUSE_RANGE = (0.1, 0.3)
    # The range (min, max) for the duration (in seconds) to sleep after each mouse movement.
    # This simulates the time a human might take to move the mouse.

    # --- Request Delays and Retries ---
    URL_DELAY_RANGE = (0.5, 1)
    # The range (min, max) for the duration (in seconds) to sleep between fetching different URLs.
    # This helps avoid overloading the server and mimics human browsing speed.

    RETRY_ATTEMPTS = 3
    # The maximum number of times to retry fetching a URL if it fails.
    # This helps handle transient network issues and improves reliability.

    TIMEOUT = 10000
    # The timeout (in milliseconds) for page navigation and element selection.
    # This prevents the scraper from getting stuck on slow-loading pages.


    VIEWPORT_SIZES = [
        {"width": 1920, "height": 1080},
        {"width": 1366, "height": 768},
        {"width": 1280, "height": 720},
        {"width": 1440, "height": 900}
    ]
    USER_AGENTS = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59",
    ]
    CONTENT_SELECTORS = [ # Add fallback selectors if any
        "div#content.content",
    ]
    LOG_FILE = "errors.log"
    OUTPUT_DIR = "." # Current directory

# --- Logging Setup ---
logging.basicConfig(filename=Config.LOG_FILE, level=logging.ERROR,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# --- Helper Functions ---
async def simulate_user_behavior(page, config: Config):
    """Simulates user scrolling and mouse movements."""
    for _ in range(random.randint(*config.SCROLL_ITERATIONS_RANGE)):
        scroll_position = random.randint(*config.SCROLL_DISTANCE_RANGE)
        await page.evaluate(f"window.scrollBy(0, {scroll_position})")
        await asyncio.sleep(random.uniform(*config.SLEEP_SCROLL_RANGE))

    for _ in range(random.randint(*config.MOUSE_MOVEMENTS_RANGE)):
        x, y = random.randint(0, 1920), random.randint(0, 1080)
        await page.mouse.move(x, y, steps=random.randint(5, 10))
        await asyncio.sleep(random.uniform(*config.SLEEP_MOUSE_RANGE))

async def extract_content(page, config: Config) -> Optional[str]:
    """Extracts content from the page using multiple selectors."""
    await asyncio.sleep(random.uniform(0.5, 1.0))  # Wait for dynamic content
    soup = BeautifulSoup(await page.content(), "html.parser")
    for selector in config.CONTENT_SELECTORS:
        content = soup.select_one(selector)
        if content:
            return content.get_text(strip=True)
    return None

async def fetch_page_content(url: str, config: Config) -> Optional[str]:
    """Fetches content from a URL with retries and error handling."""
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=True)
        try:
            page = await browser.new_page(
                user_agent=random.choice(config.USER_AGENTS),
                viewport=random.choice(config.VIEWPORT_SIZES)
            )
            await page.goto(url, timeout=config.TIMEOUT)
            await page.wait_for_selector(config.CONTENT_SELECTORS[0], timeout=config.TIMEOUT) # Wait for at least one selector to be present
            await simulate_user_behavior(page, config)
            return await extract_content(page, config)
        except TimeoutError as te:
            logging.error(f"Timeout error fetching URL {url}: {te}")
            return None
        except Exception as e:
            logging.error(f"Error fetching URL {url}: {e}")
            return None
        finally:
            if browser:
                await browser.close()

async def fetch_article_with_retries(url: str, config: Config) -> Optional[str]:
    """Fetches article content with retries."""
    for attempt in range(config.RETRY_ATTEMPTS):
        content = await fetch_page_content(url, config)
        if content:
            return content
        if attempt < config.RETRY_ATTEMPTS - 1:
            await asyncio.sleep(random.uniform(1, 3) * (attempt + 1))  # Exponential backoff
        else:
            logging.error(f"Failed to fetch URL {url} after {config.RETRY_ATTEMPTS} attempts.")
    return None

async def fetch_all_articles(df: pd.DataFrame, config: Config) -> List[Dict]:
    """Fetches content for all articles in the DataFrame."""
    data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Fetching Articles"):
        url = row["article_url"]
        content = await fetch_article_with_retries(url, config)
        if content:
            data.append({
                "article_id": row["article_id"],
                "title": row["title"],
                "url": url,
                "content": content
            })
        await asyncio.sleep(random.uniform(*config.URL_DELAY_RANGE))
    return data

def save_to_json(data: List[Dict], config: Config):
    """Saves the scraped data to a JSON file with error handling."""
    timestamp_str = time.strftime("%Y%m%d_%H%M%S")
    filename = f"articles_{timestamp_str}.json"
    filepath = os.path.join(config.OUTPUT_DIR, filename)

    try:
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Data successfully saved to: {filepath}")
    except FileNotFoundError:
        logging.error(f"Error: Output directory not found: {config.OUTPUT_DIR}")
        print(f"Error: Could not save data. Output directory not found: {config.OUTPUT_DIR}")
    except PermissionError:
        logging.error(f"Error: Permission denied to write to: {filepath}")
        print(f"Error: Could not save data. Permission denied to write to: {filepath}")
    except Exception as e:
        logging.error(f"Error saving data to JSON file: {e}")
        print(f"Error: Could not save data. An unexpected error occurred: {e}")


# --- Main Program ---
config = Config()

scraped_data = asyncio.run(fetch_all_articles(df, config))

save_to_json(scraped_data, config)

print("Scraping complete. Data saved to JSON.")



Fetching Articles:  18%|█▊        | 9/50 [01:07<05:07,  7.49s/it]